In [1]:
import os.path as op
import os
import pandas as pd
import numpy as np
from copy import deepcopy

In [2]:
data_dirs = ['/data/NKIRS-preproc-phd/aggregate_models_data/results_D100_bmi/']
files = [op.join(d, f)
         for d in data_dirs
         for f in os.listdir(d)
         if f.endswith('csv')]

In [3]:
full_results = []
for fl in files:
    tmpres = pd.read_csv(fl, index_col=0)
    fname_pieces = fl.split('_')
    dset = fname_pieces[-9].split('/')[0]
    experiment = fname_pieces[-7]
    n_mca = int(fname_pieces[-6])
    clf = fname_pieces[-4]
    dimens = fname_pieces[-2]
    preproc = fname_pieces[-3]
    random_seed = fname_pieces[-1].strip('.csv')
    resdict = {
        "Classifier": clf,
        "Experiment": experiment,
        "Simulations": n_mca,
        "Preprocessing": preproc,
        "Dimensionality": dimens,
        "Dataset": dset,
        "rs": random_seed
    }

    for idx, r in tmpres.iterrows():
        resdict_copy = deepcopy(resdict)
        # Only add ref performance for the "full" dataset
        if r.aggregation == "ref" and (n_mca != 20 or experiment != 'mca'):
            continue
        for k, v in r.items():
            resdict_copy[k] = v
        full_results += [resdict_copy]
        del resdict_copy

df_results = pd.DataFrame.from_dict(full_results)

In [4]:
n_pre = 4
n_dim = 2
n_clf = 5
n_agg_m = 6
n_agg_s = 4
n_mca = 9
n_ss = 2

n_pre*n_dim*n_clf*n_agg_m*n_mca + n_pre*n_dim*n_clf

n_pre*n_dim*n_clf*n_agg_m*n_mca + n_pre*n_dim*n_clf + n_pre*n_dim*n_clf*n_agg_s*n_ss

2520

In [5]:
df_results.query('Dataset == "D100"')

,Classifier,Experiment,Simulations,Preprocessing,Dimensionality,Dataset,rs,aggregation,acc,f1,p_acc,p_f1,test_acc,test_f1,p_test_acc,p_test_f1,test_mean_acc,test_mean_f1,n_models
0,ada,mca,10,graph,pca,D100,41,mega,0.633670,0.710585,0.003800,0.000200,0.563636,0.688312,0.02200,0.000000,0.596364,0.699113,5
1,lrc,mca,14,rankgraph,fa,D100,41,median,0.550000,0.603232,0.281200,0.352200,0.650000,0.740741,0.05900,0.045000,0.600000,0.692446,5
2,lrc,mca,20,graph,pca,D100,41,truncate,0.537500,0.606486,0.339800,0.339200,0.750000,0.814815,0.00500,0.005000,0.660000,0.715440,5
3,rfc,mca,7,graph,fa,D100,41,mega,0.593829,0.703723,0.156400,0.008200,0.618750,0.748971,0.00100,0.000000,0.626250,0.737186,5
4,svc,mca,20,rankgraph,pca,D100,41,truncate,0.640833,0.694300,0.151000,0.191600,0.600000,0.750000,0.13600,0.039000,0.660000,0.732784,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2555,rfc,mca,16,rankgraph,fa,D100,41,none,0.629844,0.709568,0.208087,0.187405,0.600625,0.733426,0.13515,0.034975,0.630000,0.742599,400
2556,rfc,mca,16,rankgraph,fa,D100,41,meta,0.617500,0.704317,0.238600,0.202200,0.646875,0.757040,0.07225,0.022812,0.638125,0.748384,5
2557,lrc,mca,10,graph,pca,D100,41,mega,0.551835,0.622667,0.194400,0.166800,0.654545,0.720588,0.00000,0.000000,0.650909,0.712713,5
2558,lrc,mca,20,zgraph,fa,D100,41,mean,0.563333,0.612222,0.292200,0.341200,0.550000,0.640000,0.24800,0.188000,0.560000,0.626531,5


In [6]:
df_results.to_csv("../data/aggregation_results_sex.csv", index=False)